In [1]:
import spacy
import yaml
import csv
from pprint import pprint as print
from scipy.stats import wasserstein_distance
import numpy as np
import copy
import pandas as pd

nlp = spacy.load("en_core_web_sm")
#spacy.explain('GPE') explains spacy abbreviations

In [2]:
def add_to_dict(doc, dict):
    for ent in doc.ents:
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        name = ent.text
        label = ent.label_
        position = ent.start_char
    
        if label != 'PERSON':
            continue
    
        if ent.text not in dict:
            dict[name] = [position]
        else:
            dict[name].append(position)
            
    return dict

In [3]:
def load_data(doc):
    with open(doc) as f:
        data = f.read()
        
    midpoint = len(data) // 2
    first_half = data[:midpoint]
    second_half = data[midpoint:]
    
    doc1 = nlp(first_half)
    doc2 = nlp(second_half)
    
    return (doc1, doc2)

In [28]:
def save_indexes(book, dict):
    with open(f'indexes/{book}.csv', 'w') as f:
        for key in dict.keys():
            value = dict[key]
            if (value.startswith('[')):
                value = value[1:]
            elif (value.endswith(']')):
                value = value[:-1]
            f.write("%s, %s\n" % (key, value))

In [10]:
def distance(character1, character2, novel_length, t):
    character1 = copy.deepcopy(character1)
    character2 = copy.deepcopy(character2)

    if len(character1) < len(character2):
        tmp = character1
        character1 = character2
        character2 = tmp

    character1new = []
    used_list = []

    j_nearest = None

    # Find a subset of appearances of the first character so that the first character is as close as possible to the
    # second character
    for i in range(len(character2)):
        min_distance = float('inf')
        for j in range(len(character1)):
            curr_distance = abs(character1[j] - character2[i])
            if curr_distance < min_distance and j not in used_list:
                min_distance = curr_distance
                j_nearest = j
        character1new.append(character1[j_nearest])
        used_list.append(j_nearest)

    character1new.sort()

    for i in range(len(character1new)):
        character1new[i] /= novel_length
        character2[i] /= novel_length

    for i in range(len(character1new)):
        character1new[i] **= (1+t)

    for i in range(len(character1new)):
        character2[i] **= (1+t)

    return wasserstein_distance(character1new, character2)

In [11]:
def save_distances(book, distances_t0, distances_tplus1, distances_tminus1, characters):
    distances_t0 = pd.DataFrame(distances_t0, index=characters, columns=characters)
    distances_tminus1 = pd.DataFrame(distances_tminus1, index=characters, columns=characters)
    distances_tplus1 = pd.DataFrame(distances_tplus1, index=characters, columns=characters)
    distances_t0.to_csv("distances/" + book + "_t0.csv")
    distances_tminus1.to_csv("distances/" + book + "_tminus1.csv")
    distances_tplus1.to_csv("distances/" + book + "_tplus1.csv")

In [20]:
dict = {}

book = 'TheBrothersKaramazov'
(doc1, doc2) = load_data('books/' + book + '.txt')
dict = add_to_dict(doc1, dict)
dict = add_to_dict(doc2, dict)

In [24]:
#CONVERTS TO LIST!
most_common_names = sorted(dict.items(), key=lambda x: len(x[1]), reverse=True)
most_common_names = most_common_names[0:10]

names_dict = {}
for item in most_common_names:
    names_dict[item[0]] = item[1]

In [52]:
save_indexes(book, names_dict)

AttributeError: 'list' object has no attribute 'startswith'

In [32]:
distances_t0 = np.zeros((10, 10))
distances_tminus1 = np.zeros((10, 10))
distances_tplus1 = np.zeros((10, 10))

In [11]:
characters = list(names_dict.keys())
novel_length = len(doc1) + len(doc2)
appearances = list(names_dict.values())

In [12]:
for i in range(9):
    for j in range(i+1, 10):
            distances_t0[i][j] = distance(appearances[i], appearances[j], novel_length, 0)
            distances_tminus1[i][j] = distance(appearances[i], appearances[j], novel_length, -0.1)
            distances_tplus1[i][j] = distance(appearances[i], appearances[j], novel_length, 0.1)
            distances_t0[j][i] = distances_t0[i][j]
            distances_tminus1[j][i] = distances_tminus1[i][j]
            distances_tplus1[j][i] = distances_tplus1[i][j]

In [13]:
save_distances(book, distances_t0, distances_tplus1, distances_tminus1, characters)